# IRGAN

Codigo:

https://github.com/geek-ai/irgan

paper: 

https://arxiv.org/pdf/1705.10513.pdf

Una GAN discreta (como SeqGan) pero que el generador es modelado como un politica de aprendizaje reforzado para seleccionar el documento candidato $d$ en el estado dado la query $q_n$, y una metrica de relevancia $r$ que es entrenado via una politica de gradientes

Quieren poner enfoque en el entrenamiento adversario, osea usan modelos simples e iguales para D y G

El módulo pickle implementa un algoritmo para convertir un objeto Python arbitrario en una serie de bytes. Este proceso también se llama serializar "el objeto". El flujo de bytes que representa el objeto se puede transmitir o almacenar y luego reconstruir para crear un nuevo objeto con las mismas características.

El módulo cPickle implementa el mismo algoritmo, en C en lugar de Python. Es muchas veces más rápido que la implementación de Python, pero no permite al usuario crear una subclase de Pickle. Si la subclasificación no es importante para su uso, probablemente desee utilizar cPickle.

In [ ]:
import _pickle as cPickle #guarda el modelo..
from IPython.display import Latex
import tensorflow as tf
import numpy as np
import multiprocessing
import pandas as pd 
import random
import pickle

cores = multiprocessing.cpu_count()
random.seed(0)


import matplotlib.pyplot as plt
import heapq
tf.compat.v1.disable_eager_execution()

## Discriminativo DNS

In [ ]:

class DIS():
    def __init__(self, itemNum, userNum, emb_dim,visual_matrix, visual_emb,reviews, lamda, param=None, initdelta=0.05, learning_rate=0.05,imageFeatureDim= 4096):
        self.itemNum = itemNum
        self.userNum = userNum
        self.emb_dim = emb_dim
        self.lamda = lamda  # regularization parameters
        self.param = param
        self.visual_I = visual_matrix #preentrenada no aprendible
        self.k2 = visual_emb
        self.initdelta = initdelta
        self.learning_rate = learning_rate
        self.reviews = reviews #[usernum,1]
        self.imageFeatureDim= imageFeatureDim
        self.d_params = []

        with tf.compat.v1.compat.v1.variable_scope('discriminator'):
            if self.param is None: #Si no hay param -> inicializacion aleatoria
                self.user_embeddings = tf.compat.v1.Variable(
                    tf.compat.v1.random_uniform([self.userNum, self.emb_dim], minval=-self.initdelta, maxval=self.initdelta,
                                      dtype=tf.compat.v1.float32))
                self.item_embeddings = tf.compat.v1.Variable(
                    tf.compat.v1.random_uniform([self.itemNum, self.emb_dim], minval=-self.initdelta, maxval=self.initdelta,
                                      dtype=tf.compat.v1.float32))
                self.item_bias = tf.compat.v1.Variable(tf.compat.v1.zeros([self.itemNum]))
                #####################################
                self.visual_U = tf.compat.v1.Variable(
                    tf.compat.v1.random_uniform([self.userNum, self.k2], minval=-self.initdelta, maxval=self.initdelta,
                                      dtype=tf.compat.v1.float32))
                #E
                self.itemEmb_W = tf.compat.v1.Variable(
                    tf.compat.v1.random_uniform([self.imageFeatureDim,self.k2], minval=-self.initdelta, maxval=self.initdelta,
                                      dtype=tf.compat.v1.float32))
                #beta
                self.visual_item_bias = tf.compat.v1.Variable(
                    tf.compat.v1.random_uniform([self.itemNum,self.imageFeatureDim], minval=-self.initdelta, maxval=self.initdelta,
                                      dtype=tf.compat.v1.float32))
                

                ###################################################
                self.bias_reviews = tf.compat.v1.Variable(tf.compat.v1.zeros([self.userNum]))

            else: #lee los parametros del archivo
                self.user_embeddings = tf.compat.v1.Variable(self.param[0])
                self.item_embeddings = tf.compat.v1.Variable(self.param[1])
                self.item_bias = tf.compat.v1.Variable(self.param[2])
                #######################################
                self.visual_U = tf.compat.v1.Variable(self.param[3])
                self.itemEmb_W = tf.compat.v1.Variable(self.param[4])
                self.visual_item_bias = tf.compat.v1.Variable(self.param[5])
                ###########################
                self.bias_reviews = tf.compat.v1.Variable(self.param[6])

            self.d_params = [self.user_embeddings, self.item_embeddings, self.item_bias,  self.visual_U, self.itemEmb_W, self.visual_item_bias , self.bias_reviews]
        
        # placeholder definition
        self.u = tf.compat.v1.placeholder(tf.compat.v1.int32)
        self.pos = tf.compat.v1.placeholder(tf.compat.v1.int32)
        self.neg = tf.compat.v1.placeholder(tf.compat.v1.int32)

        self.u_embedding = tf.compat.v1.nn.embedding_lookup(self.user_embeddings, self.u)
        self.pos_embedding = tf.compat.v1.nn.embedding_lookup(self.item_embeddings, self.pos)
        self.pos_bias = tf.compat.v1.gather(self.item_bias, self.pos)
        self.neg_embedding = tf.compat.v1.nn.embedding_lookup(self.item_embeddings, self.neg)
        self.neg_bias = tf.compat.v1.gather(self.item_bias, self.neg)

        ##################################################################################

        self.visual_U_vector = tf.compat.v1.nn.embedding_lookup(self.visual_U, self.u)
        self.visual_I_matrix_pos = tf.compat.v1.nn.embedding_lookup(self.visual_I, self.pos) #[1,4096]
        self.visual_I_matrix_neg = tf.compat.v1.nn.embedding_lookup(self.visual_I, self.neg)
        self.visual_pos_bias = tf.compat.v1.gather(self.visual_item_bias, self.pos) #[1,4096]
        self.visual_neg_bias = tf.compat.v1.gather(self.visual_item_bias, self.neg)

       ##################

        self.Ef_pos= tf.matmul( self.visual_I_matrix_pos,self.itemEmb_W) #f_i x E= [1,4096]x[4096,k2]= [1,k2]
        self.Ef_neg= tf.matmul(self.visual_I_matrix_neg,self.itemEmb_W) 
        self.Ef= tf.compat.v1.matmul(self.itemEmb_W,self.visual_I,transpose_a=True,transpose_b=True ) #E^T x f^T =  [k2,4096]x[4096,#I] = [k2,#I]
        self.bf_pos=tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.visual_I_matrix_pos, self.visual_pos_bias),1)#[1,4096]x[1,4096]
        self.bf_neg=  tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.visual_I_matrix_neg, self.visual_neg_bias),1)
          
        
        ##################

        self.reviews_u = tf.compat.v1.gather(self.reviews, self.u) #[#dim_u,1]
        self.dim_u= tf.shape(self.u_embedding)[0]
        self.b= tf.compat.v1.gather(self.bias_reviews, self.u) #[1, #dim_u]
        self.ones= tf.ones([self.itemNum,self.dim_u]) #[#I,dim_u]
        self.sxb_s= tf.compat.v1.multiply(self.b, tf.transpose(self.reviews_u) ) #[1,#dimu]x[1,#dim_u]= [1,#dimu]
        self.user_bias_review = tf.transpose(tf.compat.v1.multiply(self.sxb_s, self.ones)) #[#dim_u,#I]

        self.pre_logits = tf.compat.v1.sigmoid(
            tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.u_embedding, self.pos_embedding - self.neg_embedding),1) # [1,k]x[1,k] multuply es element wise, reduce sum sum, osea producto punto.
             + self.pos_bias - self.neg_bias
             +  tf.compat.v1.reduce_sum( tf.compat.v1.multiply(self.visual_U_vector,self.Ef_pos- self.Ef_neg ) ,1 )  # [1,k2]x[1,k2]
             +self.bf_pos- self.bf_neg 
                          ) +self.sxb_s
             
        
 
        self.reg= self.lamda * (
            tf.compat.v1.nn.l2_loss(self.u_embedding) +
            tf.compat.v1.nn.l2_loss(self.pos_embedding) +
            tf.compat.v1.nn.l2_loss(self.pos_bias) +
            tf.compat.v1.nn.l2_loss(self.neg_embedding) +
            tf.compat.v1.nn.l2_loss(self.neg_bias)+
            tf.compat.v1.nn.l2_loss(self.visual_U_vector) +
            tf.compat.v1.nn.l2_loss(self.itemEmb_W) +
            tf.compat.v1.nn.l2_loss(self.visual_neg_bias) +
            tf.compat.v1.nn.l2_loss(self.visual_pos_bias)+
            tf.compat.v1.nn.l2_loss(self.b)
            )

        self.pre_loss = -tf.compat.v1.reduce_mean(tf.compat.v1.log(self.pre_logits)) + self.reg
        d_opt = tf.compat.v1.train.GradientDescentOptimizer(self.learning_rate)
        self.d_updates = d_opt.minimize(self.pre_loss, var_list=self.d_params)

        # for test stage, self.u: [batch_size]
        
                          # uxI= [batch_size,k]x[k,#I]=[batch_size,#I]                                                     # b_i= [1, #I]        # [batch_size, k2]x[k2, #I]  = [bact_size,#I]                                             #beta elementwise f = [#,4096]elementwise[#I,4096]= [1, #I]                              #[batch,#I]
        self.all_rating = tf.compat.v1.matmul(self.u_embedding, self.item_embeddings, transpose_a=False,transpose_b=True) + self.item_bias + tf.compat.v1.matmul(self.visual_U_vector, self.Ef, transpose_a=False, transpose_b=False) +  tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.visual_item_bias , self.visual_I),1)+ self.user_bias_review                    

        #self.all_logits = tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.u_embedding, self.item_embeddings), 1) + self.item_bias
        # for dns sample
        #esto da muy grande
                                                                        # [1,k]x[#I,k] = [1,#I]                                 #[1, #I]                                                      #[1,k2]x[#I,k2]=[1,#I]
        self.dns_rating = tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.u_embedding, self.item_embeddings), 1) + self.item_bias + tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.visual_U_vector, tf.transpose(self.Ef) ), 1) + tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.visual_item_bias , self.visual_I),1  )+ self.user_bias_review  

    def save_model(self, sess, filename):
        param = sess.run(self.d_params)
        cPickle.dump(param, open(filename, 'wb'))  #antes w

## utils.py

In [ ]:
import linecache
import numpy as np


def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1


# Get batch data from training set
def get_batch_data(file, index, size):  # 1,5->1,2,3,4,5
    user = []
    item = []
    label = []
    for i in range(index, index + size):
        line = linecache.getline(file, i)
        line = line.strip()
        line = line.split()
        user.append(int(line[0]))
        user.append(int(line[0]))
        item.append(int(line[1]))
        item.append(int(line[2]))
        label.append(1.)
        label.append(0.)
    return user, item, label




## Generador

In [ ]:
class GEN():
    def __init__(self, itemNum, userNum, emb_dim, visual_matrix, visual_emb, reviews,lamda, param=None, initdelta=0.05, learning_rate=0.05,imageFeatureDim= 4096):
        self.itemNum = itemNum
        self.userNum = userNum
        self.emb_dim = emb_dim
        self.lamda = lamda  # regularization parameters
        self.param = param
        self.initdelta = initdelta
        self.learning_rate = learning_rate
        self.g_params = []
        self.reviews = reviews #[usernum,1]
        self.visual_I = visual_matrix #preentrenada no aprendible
        self.k2 = visual_emb
        self.imageFeatureDim= imageFeatureDim

        with tf.compat.v1.variable_scope('generator'):
            if self.param == None: #inicialización de representaciones
                self.user_embeddings = tf.compat.v1.Variable(
                    tf.compat.v1.random_uniform([self.userNum, self.emb_dim], minval=-self.initdelta, maxval=self.initdelta,
                                      dtype=tf.compat.v1.float32))
                self.item_embeddings = tf.compat.v1.Variable(
                    tf.compat.v1.random_uniform([self.itemNum, self.emb_dim], minval=-self.initdelta, maxval=self.initdelta,
                                      dtype=tf.compat.v1.float32))
                self.item_bias = tf.compat.v1.Variable(tf.compat.v1.zeros([self.itemNum]))
                #####################################
                self.visual_U = tf.compat.v1.Variable(
                    tf.compat.v1.random_uniform([self.userNum, self.k2], minval=-self.initdelta, maxval=self.initdelta,
                                      dtype=tf.compat.v1.float32))
                #E
                self.itemEmb_W = tf.compat.v1.Variable(
                    tf.compat.v1.random_uniform([self.imageFeatureDim,self.k2], minval=-self.initdelta, maxval=self.initdelta,
                                      dtype=tf.compat.v1.float32))
                #beta
                self.visual_item_bias = tf.compat.v1.Variable(
                    tf.compat.v1.random_uniform([self.itemNum,self.imageFeatureDim], minval=-self.initdelta, maxval=self.initdelta,
                                      dtype=tf.compat.v1.float32))

                ###################################################
                self.bias_reviews = tf.compat.v1.Variable(tf.compat.v1.zeros([self.userNum]))

            else:
                self.user_embeddings = tf.compat.v1.Variable(self.param[0])
                self.item_embeddings = tf.compat.v1.Variable(self.param[1])
                self.item_bias = tf.compat.v1.Variable(param[2])
                                #######################################
                self.visual_U = tf.compat.v1.Variable(self.param[3])
                self.itemEmb_W = tf.compat.v1.Variable(self.param[4])
                self.visual_item_bias = tf.compat.v1.Variable(self.param[5])
                ###########################
                self.bias_reviews = tf.compat.v1.Variable(self.param[6])
            self.g_params = [self.user_embeddings, self.item_embeddings, self.item_bias,self.visual_U, self.itemEmb_W, self.visual_item_bias, self.bias_reviews ]

        self.u = tf.compat.v1.placeholder(tf.compat.v1.int32)
        self.i = tf.compat.v1.placeholder(tf.compat.v1.int32)
        self.reward = tf.compat.v1.placeholder(tf.compat.v1.float32)

        self.u_embedding = tf.compat.v1.nn.embedding_lookup(self.user_embeddings, self.u)
        self.i_embedding = tf.compat.v1.nn.embedding_lookup(self.item_embeddings, self.i)
        self.i_bias = tf.compat.v1.gather(self.item_bias, self.i)


        ##################################################################################

        self.visual_U_vector = tf.compat.v1.nn.embedding_lookup(self.visual_U, self.u)
        self.visual_I_matrix_i= tf.compat.v1.nn.embedding_lookup(self.visual_I, self.i) #[1,4096]
        self.visual_bias_i = tf.compat.v1.gather(self.visual_item_bias, self.i) #[1,4096]

       ##################

        self.Ef_i= tf.matmul( self.visual_I_matrix_i,self.itemEmb_W) #f_i x E= [1,4096]x[4096,k2]= [1,k2]
        self.Ef= tf.compat.v1.matmul(self.itemEmb_W,self.visual_I,transpose_a=True,transpose_b=True ) #E^T x f^T =  [k2,4096]x[4096,#I] = [k2,#I]
        self.bf_i=tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.visual_I_matrix_i, self.visual_bias_i),1)#[1,4096]x[1,4096]
          
        
        ##################

        self.reviews_u = tf.compat.v1.gather(self.reviews, self.u) #[#dim_u,1]
        self.dim_u= tf.shape(self.u_embedding)[0]
        self.b= tf.compat.v1.gather(self.bias_reviews, self.u) #[1, #dim_u]
        self.ones= tf.ones([self.itemNum,self.dim_u]) #[#I,dim_u]
        self.sxb_s= tf.compat.v1.multiply(self.b, tf.transpose(self.reviews_u) ) #[1,#dimu]x[1,#dim_u]= [1,#dimu]
        self.user_bias_review = tf.transpose(tf.compat.v1.multiply(self.sxb_s, self.ones)) #[#dim_u,#I]



                                                                        #[1,k]x[#I,k]                                   #[1,#I]                                # [1,k2]x[1,k2]
        self.all_logits = tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.u_embedding, self.item_embeddings), 1) + self.item_bias +  tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.visual_U_vector, tf.transpose(self.Ef) ), 1) + tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.visual_item_bias , self.visual_I),1)+self.sxb_s  

        self.i_prob = tf.compat.v1.gather(
            tf.compat.v1.reshape(tf.compat.v1.nn.softmax(tf.compat.v1.reshape(self.all_logits, [1, -1])), [-1]),
            self.i)

        self.reg= self.lamda * (tf.compat.v1.nn.l2_loss(self.u_embedding) + tf.compat.v1.nn.l2_loss(self.i_embedding) + tf.compat.v1.nn.l2_loss(self.i_bias)+
                                 tf.compat.v1.nn.l2_loss(self.visual_U_vector) +tf.compat.v1.nn.l2_loss(self.itemEmb_W) +tf.compat.v1.nn.l2_loss(self.visual_bias_i)  + tf.compat.v1.nn.l2_loss(self.b) )
        
        self.gan_loss = -tf.compat.v1.reduce_mean(tf.compat.v1.log(self.i_prob) * self.reward) + self.reg

        g_opt = tf.compat.v1.train.GradientDescentOptimizer(self.learning_rate)

        self.gan_updates = g_opt.minimize(self.gan_loss, var_list=self.g_params)

        # for test stage, self.u: [batch_size]
        self.all_rating = tf.compat.v1.matmul(self.u_embedding, self.item_embeddings, transpose_a=False,
                                    transpose_b=True) + self.item_bias + tf.compat.v1.matmul(self.visual_U_vector, self.Ef, transpose_a=False, transpose_b=False) +  tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.visual_item_bias , self.visual_I),1) + self.user_bias_review 

    def save_model(self, sess, filename):
        param = sess.run(self.g_params)
        cPickle.dump(param, open(filename, 'wb')) #mismo cambio

## Discriminador

$f_{\phi} (q,d)= s_{\phi} (q,d)= s(u,i)= b_i + v_u^Tv_i$

Máquina de factorización sencilla

Su labor es distinguir (clasificar) si el documento $d$ es relevante (real, etiqueta=1, ejemplo positivo) para la consulta $q$. Estimando la probabilidad:

$D(d|q)= \sigma (f_{\phi} (q,d))= \dfrac{exp(f_{\phi} (q,d))}{1+ exp(f_{\phi} (q,d))} $

Su objetivo entonces es maximiza la log verosimititud de distrinción entre documentos verdaderos y generados:

$\phi^{*}=\arg \max _{\phi} \sum_{n=1}^{N}\left(\mathbb{E}_{d \sim p_{\text {true }}\left(d \mid q_{n}, r\right)}\left[\log \left(\sigma\left(f_{\phi}\left(d, q_{n}\right)\right)\right]+\right.\right.$
$\left.\mathbb{E}_{d \sim p_{\theta^{*}}\left(d \mid q_{n}, r\right)}\left[\log \left(1-\sigma\left(f_{\phi}\left(d, q_{n}\right)\right)\right)\right]\right)$

Modelo generativo actual fijo

In [ ]:
class DIS():
    def __init__(self, itemNum, userNum, emb_dim, visual_matrix, visual_emb,reviews, lamda, param=None, initdelta=0.05, learning_rate=0.05,imageFeatureDim=4096):
        self.itemNum = itemNum
        self.userNum = userNum
        self.emb_dim = emb_dim
        self.lamda = lamda  # regularization parameters
        self.param = param
        self.reviews = reviews #[usernum,1]
        self.initdelta = initdelta
        self.learning_rate = learning_rate
        self.d_params = []
        self.visual_I = visual_matrix #preentrenada no aprendible
        self.k2 = visual_emb
        self.imageFeatureDim= imageFeatureDim

        with tf.compat.v1.variable_scope('discriminator'):
          #inicializacion
            if self.param == None:
                self.user_embeddings = tf.compat.v1.Variable(
                    tf.compat.v1.random_uniform([self.userNum, self.emb_dim], minval=-self.initdelta, maxval=self.initdelta,
                                      dtype=tf.compat.v1.float32))
                self.item_embeddings = tf.compat.v1.Variable(
                    tf.compat.v1.random_uniform([self.itemNum, self.emb_dim], minval=-self.initdelta, maxval=self.initdelta,
                                      dtype=tf.compat.v1.float32))
                self.item_bias = tf.compat.v1.Variable(tf.compat.v1.zeros([self.itemNum]))
                #####################################
                self.visual_U = tf.compat.v1.Variable(
                    tf.compat.v1.random_uniform([self.userNum, self.k2], minval=-self.initdelta, maxval=self.initdelta,
                                      dtype=tf.compat.v1.float32))
                #E
                self.itemEmb_W = tf.compat.v1.Variable(
                    tf.compat.v1.random_uniform([self.imageFeatureDim,self.k2], minval=-self.initdelta, maxval=self.initdelta,
                                      dtype=tf.compat.v1.float32))
                #beta
                self.visual_item_bias = tf.compat.v1.Variable(
                    tf.compat.v1.random_uniform([self.itemNum,self.imageFeatureDim], minval=-self.initdelta, maxval=self.initdelta,
                                      dtype=tf.compat.v1.float32))

                ###################################################
                self.bias_reviews = tf.compat.v1.Variable(tf.compat.v1.zeros([self.userNum]))

            else:
                self.user_embeddings = tf.compat.v1.Variable(self.param[0])
                self.item_embeddings = tf.compat.v1.Variable(self.param[1])
                self.item_bias = tf.compat.v1.Variable(self.param[2])
                #######################################
                self.visual_U = tf.compat.v1.Variable(self.param[3])
                self.itemEmb_W = tf.compat.v1.Variable(self.param[4])
                self.visual_item_bias = tf.compat.v1.Variable(self.param[5])
                ###########################
                self.bias_reviews = tf.compat.v1.Variable(self.param[6])                

        self.d_params = [self.user_embeddings, self.item_embeddings, self.item_bias,self.visual_U, self.itemEmb_W, self.visual_item_bias , self.bias_reviews ]

        # placeholder definition
        self.u = tf.compat.v1.placeholder(tf.compat.v1.int32)
        self.i = tf.compat.v1.placeholder(tf.compat.v1.int32)
        self.label = tf.compat.v1.placeholder(tf.compat.v1.float32)
        #indexaciones
        #V_u
        self.u_embedding = tf.compat.v1.nn.embedding_lookup(self.user_embeddings, self.u)
        #V_i
        self.i_embedding = tf.compat.v1.nn.embedding_lookup(self.item_embeddings, self.i)
        #b_i
        self.i_bias = tf.compat.v1.gather(self.item_bias, self.i)


        ##################################################################################

        self.visual_U_vector = tf.compat.v1.nn.embedding_lookup(self.visual_U, self.u)
        self.visual_I_matrix_i= tf.compat.v1.nn.embedding_lookup(self.visual_I, self.i) #[1,4096]
        self.visual_bias_i = tf.compat.v1.gather(self.visual_item_bias, self.i) #[1,4096]

       ##################
        self.Ef_i= tf.matmul( self.visual_I_matrix_i,self.itemEmb_W) #f_i x E= [1,4096]x[4096,k2]= [1,k2]
        self.Ef= tf.compat.v1.matmul(self.itemEmb_W,self.visual_I,transpose_a=True,transpose_b=True ) #E^T x f^T =  [k2,4096]x[4096,#I] = [k2,#I]
        self.bf_i=tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.visual_I_matrix_i, self.visual_bias_i),1)#[1,4096]x[1,4096]
          
        
        ##################
        
        self.reviews_u = tf.compat.v1.gather(self.reviews, self.u) #[#dim_u,1]
        self.dim_u= tf.shape(self.u_embedding)[0]
        self.b= tf.compat.v1.gather(self.bias_reviews, self.u) #[1, #dim_u]
        self.ones= tf.ones([self.itemNum,self.dim_u]) #[#I,dim_u]
        self.sxb_s= tf.compat.v1.multiply(self.b, tf.transpose(self.reviews_u) ) #[1,#dimu]x[1,#dim_u]= [1,#dimu]
        self.user_bias_review = tf.transpose(tf.compat.v1.multiply(self.sxb_s, self.ones)) #[#dim_u,#I]
        #################

        #b_i+vu*v_i
        self.pre_logits = tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.u_embedding, self.i_embedding), 1) + self.i_bias + tf.compat.v1.reduce_sum( tf.compat.v1.multiply(self.visual_U_vector,self.Ef_i ) ,1 ) + self.bf_i+ self.sxb_s
        #sigmoide cross entropy con regularización l2 
        self.reg= self.lamda * (
            tf.compat.v1.nn.l2_loss(self.u_embedding) + tf.compat.v1.nn.l2_loss(self.i_embedding) + tf.compat.v1.nn.l2_loss(self.i_bias)+
                                 tf.compat.v1.nn.l2_loss(self.visual_U_vector) +tf.compat.v1.nn.l2_loss(self.itemEmb_W) +tf.compat.v1.nn.l2_loss(self.visual_bias_i)+ tf.compat.v1.nn.l2_loss(self.b) )
        
        self.pre_loss = tf.compat.v1.nn.sigmoid_cross_entropy_with_logits(labels=self.label,
                                                                logits=self.pre_logits) + self.reg
        #SGD
        d_opt = tf.compat.v1.train.GradientDescentOptimizer(self.learning_rate)
        self.d_updates = d_opt.minimize(self.pre_loss, var_list=self.d_params)
        #Recompensa: 
        #b_i+vu*v_i
        self.reward_logits = tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.u_embedding, self.i_embedding),
                                           1) + self.i_bias +  tf.compat.v1.reduce_sum( tf.compat.v1.multiply(self.visual_U_vector,self.Ef_i ) ,1 ) + self.bf_i+ self.sxb_s
        #2sig(f)-1
        self.reward = 2 * (tf.compat.v1.sigmoid(self.reward_logits) - 0.5)

        # for test stage, self.u: [batch_size]
        self.all_rating = tf.compat.v1.matmul(self.u_embedding, self.item_embeddings, transpose_a=False,
                                    transpose_b=True) + self.item_bias + tf.compat.v1.matmul(self.visual_U_vector, self.Ef, transpose_a=False, transpose_b=False) +  tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.visual_item_bias , self.visual_I),1) +self.user_bias_review  

        #1 usuario todos los items.
        
        self.all_logits = tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.u_embedding, self.item_embeddings), 1) + self.item_bias +  tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.visual_U_vector, tf.transpose(self.Ef) ), 1) + tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.visual_item_bias , self.visual_I),1) +self.user_bias_review   
        
        #self.NLL = -tf.compat.v1.reduce_mean(tf.compat.v1.log(
            #tf.compat.v1.gather(tf.compat.v1.reshape(tf.compat.v1.nn.softmax(tf.compat.v1.reshape(self.all_logits, [1, -1])), [-1]), self.i)))
        # for dns sample
         
        self.dns_rating = tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.u_embedding, self.item_embeddings), 1) + self.item_bias + tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.visual_U_vector, tf.transpose(self.Ef) ), 1) + tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.visual_item_bias , self.visual_I),1)  +self.user_bias_review  

    def save_model(self, sess, filename):
        param = sess.run(self.d_params)
        cPickle.dump(param, open(filename, 'w'))

# Adaptación de IRGAN

## Lectura Amazon 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

workdir = '/content/drive/MyDrive/dataset_ml/amazon_men_actualizado/' 
#workdir = '/content/drive/MyDrive/dataset_ml/women/' 

usuarios_train = np.load(workdir+'dic_train_men.npy',allow_pickle='TRUE')
usuarios_train=list(usuarios_train.reshape(-1,1))[0][0]
usuarios_test = np.load(workdir+'dic_test_men.npy',allow_pickle='TRUE')
usuarios_test=list(usuarios_test.reshape(-1,1))[0][0]
DIS_TRAIN_FILE =workdir+ "dis-train.txt"
DIS_MODEL_FILE =   workdir+"model_dns_IRGANvisual.pkl" #rutaaaa

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dic_feautures = np.load(workdir+'features_women_effnet.npy',allow_pickle='TRUE')
dic_feautures=list(dic_feautures.reshape(-1,1))[0][0]

In [ ]:
#########################################################################################
# Hyper-parameters
#########################################################################################
EMB_DIM = 10
K2=500
DNS_K = 5

In [ ]:
user_pos_test = {}
user_pos_train_review = {}
user_pos_train= {}
c_t=0
all_items= []


for u in usuarios_train.keys():
  for item in usuarios_train[u]:
    if item[b"productid"] not in all_items:
      all_items.append(item[b"productid"])
    
    if u not in user_pos_train: #nuevo
      user_pos_train[u] = [item[b"productid"]]
      user_pos_train_review[u]= [item["reviewText"]]
    else: #sino agrego
      user_pos_train[u].append(item[b"productid"])
      user_pos_train_review[u].append(item["reviewText"])
all_user= list(user_pos_train.keys())

for u in usuarios_train.keys():    
  item=usuarios_test[u][0]
  if item[b"productid"] not in all_items:
    c_t+=1
  else:
    user_pos_test[u] = [item[b"productid"]]

ITEM_NUM=len(all_items)
USER_NUM= len(usuarios_train.keys())

In [ ]:
print("% de items en test que no estan en train: ",c_t/len(all_items)*100)
print("n item: ", ITEM_NUM, "n usuarios: ", USER_NUM)

% de items en test que no estan en train:  2.888205578739293
n item:  9106 n usuarios:  3834


In [ ]:
c=0
for i in all_items:
  vector= dic_feautures[i]
  if c==0:
    visual_matrix= vector
    c=1
  else:
    visual_matrix= np.concatenate((visual_matrix, vector),axis=0)

NameError: ignored

In [ ]:
print(visual_matrix.shape)

(7245, 1536)


In [ ]:
np.max(visual_matrix)

19.334303

In [ ]:
visual_matrix= visual_matrix/np.max(visual_matrix)

#men

In [ ]:
from textblob import TextBlob
reviw_prom={}
for u in user_pos_train_review.keys():
  l=[]
  for item in user_pos_train_review[u]:
     l.append(TextBlob(item).sentiment.polarity)
  reviw_prom[u]=np.mean(l)
reviews= np.reshape(list(reviw_prom.values()),(-1,1))
reviews=reviews.astype("float32")
reviews.shape

(3834, 1)

In [ ]:
reviw_prom_men= reviw_prom

In [ ]:
user_pos_train_review[all_user[ np.argmax(list(reviw_prom.values())) ]]

['great design! I wish we had this one when I was a kid!',
 'Great shirt! the teenager who is sporting it is very happy!']

In [ ]:
all_user[ np.argmax(list(reviw_prom.values())) ]

11467

['¡buen diseño! ¡Ojalá tuviéramos este cuando era niño! ',
  '¡Gran camisa! ¡el adolescente que lo está luciendo está muy feliz! ']

In [ ]:
user_pos_train_review[all_user[ np.argpartition((list(reviw_prom.values())),3)[0] ]][0]

"We are Seattle Seahawks Season ticket holders and the November/December games present some issues when it comes to keeping warm. Even though my wife and I wear parkas and jeans and bring blankets, our legs still get cold. Silk long johns are the answer. They provide as much warmth as the heavier cotton types but barely add any bulk. They're comfortable and while after sitting for a couple of hours out in the cold of the stadium our legs still get cold, these help a lot."

["Somos poseedores de boletos de temporada de los Seattle Seahawks y los juegos de noviembre / diciembre presentan algunos problemas cuando se trata de mantenernos calientes. Aunque mi esposa y yo usamos parkas y jeans y llevamos mantas, nuestras piernas todavía se enfrían. Los calzoncillos largos de seda son los respuesta. Proporcionan tanta calidez como los tipos de algodón más pesados, pero apenas agregan volumen. Son cómodos y, aunque después de estar sentados un par de horas en el frío del estadio, nuestras piernas todavía se enfrían, esto ayuda mucho ". ,
  Los uso con mis calcetines de lana Carhart. Vamos a muchas actividades al aire libre en el invierno donde tenemos que sentarnos durante unas horas y mis pies siempre se enfrían. Ya no. Funcionan a las mil maravillas '].

In [ ]:
user_pos_train_review[all_user[ np.argpartition((list(reviw_prom.values())),3)[0] ]][1]

'I use these with my Carhart Wool socks. We go to a lot of outdoor activities in the winter where we have to sit for a few hours and my feet always got cold. Not anymore. These work like a charm.'

In [ ]:
all_user[ np.argpartition((list(reviw_prom.values())),3)[0] ]

31564

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Box(y=list(reviw_prom_men.values()), name= " ") ]  )
fig.update_layout(
   title='Polaridad promedio de review por usuario',height=600, width=800)
fig.show()


#women

In [ ]:
from textblob import TextBlob
reviw_prom_w={}
for u in user_pos_train_review.keys():
  l=[]
  for item in user_pos_train_review[u]:
     l.append(TextBlob(item).sentiment.polarity)
  reviw_prom_w[u]=np.mean(l)
reviews= np.reshape(list(reviw_prom_w.values()),(-1,1))
reviews=reviews.astype("float32")
reviews.shape

(3158, 1)

In [ ]:
user_pos_train_review[all_user[ np.argmax(list(reviw_prom.values())) ]]

['The color is nice youthfull, and the style too .I bought it for my daughter and she was very happy with it.',
 'It is very sexy blouse.fits real nice!!',
 'Have been buying for about 2 years now.Love the all the blouses!!.They are very nice quality and wash great!!Just love them!!']

In [ ]:
user_pos_train_review[all_user[ np.argpartition((list(reviw_prom.values())),3)[0] ]]

["I was hoping I wouldn't have to layer with this tee, but I do. I hate it when my bra is visible through my shirt. Like the color though.",
 'Material is medium weight and stretchy. The color is true and the shirt seems well made. Fits me well. No complaints.']

In [ ]:
reviw_prom_w= reviw_prom

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Box(y=list(reviw_prom_w.values()), name= "Women")  )
fig.add_trace(go.Box(y=list(reviw_prom_men.values()), name= "Men")   )
fig.update_layout(
   title='Polaridad promedio de review por usuario',height=600, width=800)
fig.show()


# boots

## Inicialiazacion DNS

In [ ]:
DIS_TRAIN_FILE =workdir+ "dis-train_boots_text.txt"
DIS_MODEL_FILE =   workdir+"model_dns_IRGANvisual_boots_text.pkl" #rutaaaa

In [ ]:
def AUC(rating, test_users,dict_tov):
  """
  para cada usuario cuenta cuantos items de una muestra aleatoria tiene rating menor a 
  al rating del item consumido en test
  """
  ans=0
  cc=0
  for user in test_users:
      user_ind= test_users.index(user)
      user_= all_user[user]
      if dict_tov[user_][0] in all_items:
          item_test= list(all_items).index(dict_tov[user_][0])
          cc+=1
          items_train= [ list(all_items).index(i) for i in user_pos_train[user_]]  
          no_considerar= set(items_train+ [item_test]) 
          count=0
          tmpans=0 
          for j in random.sample(range(ITEM_NUM),int(50*(len(no_considerar)-1))): #sample
              if j in no_considerar: continue
              if rating[user_ind,item_test]>rating[user_ind,j]: tmpans+=1
              count+=1

          tmpans/=float(count)
          ans+=tmpans
  
  ans/=float(cc)
  return ans



def simple_test(sess, model, dict_tov): 
  #Calcula AUC para todos los usuarios comparada con el elemento que quedo en el test set
  #dict_tov: test o validacion
 test_users=[ all_user.index(i) for i in dict_tov.keys()]
 user_batch_rating = sess.run(model.all_rating, {model.u: test_users } ) 
 batch_result= AUC(user_batch_rating,test_users, dict_tov)
    
 return batch_result

In [ ]:
tf.compat.v1.reset_default_graph()
param = None
discriminator = DIS(ITEM_NUM, USER_NUM, EMB_DIM, visual_matrix, K2, reviews, lamda=0.001, param=param, initdelta=0.1, learning_rate=0.1, imageFeatureDim=1536) #se llama al discriminador DNS


config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
sess.run(tf.compat.v1.global_variables_initializer())


no sirve considerar todos ni tomar el top algo sobre una muestra mas grande

In [ ]:
%%time
dis_log = open(workdir+'dis_log_dns_IRGANvisual_text.txt', 'w')
best_AUC=simple_test(sess, discriminator, user_pos_train)
#print(sess.run(discriminator.pre_loss,feed_dict={discriminator.u: [0], discriminator.pos: [1],discriminator.neg: [2]}))
#print("auc discriminador inicializado random", best_AUC)
losses=[]
DNS_K= 5
aucs=[]
for epoch in range(int(3*13000)): 
        u= np.random.choice(list(user_pos_train.keys()),1)[0]
        i = list(all_items).index(np.random.choice(user_pos_train[u],1)[0] ) #lo que consumio
        pos = user_pos_train[u]
        all_rating = sess.run(discriminator.dns_rating, {discriminator.u: all_user.index(u)}) #evalua el modelo
        all_rating = np.array(all_rating)
        neg = []
        candidates = list(set(all_items) - set(pos) )#los que no ha consumido
        
        choice = np.random.choice(candidates, DNS_K) #se escogen DN_K aleatoriamente de los candidatos
        choice= [list(all_items).index(i) for i in choice]
        #choice_score = all_rating[choice] # sus rating
        #top_10_ind=random.sample(heapq.nlargest(5, range(len(choice_score)), choice_score.take),1)
        #j= top_10_ind[0]
        j= np.random.choice(choice,1)[0]
        u= all_user.index(u)

              #se actualiza el discriminador para la nueva tripleta
        _ ,loss= sess.run([discriminator.d_updates, discriminator.pre_loss ],
                             feed_dict={discriminator.u: [u], discriminator.pos: [i],
                                        discriminator.neg: [j]})
                
        if epoch%1000==0:
          losses.append(loss)
          AUC_actual = simple_test(sess, discriminator,user_pos_train) #evalua
          print ("epoch ", epoch, "dis: ", AUC_actual, "loss: ",loss )
          aucs.append(AUC_actual)
            
          if AUC_actual> best_AUC:
                  print("mejore")
                  best_AUC = AUC_actual
                  discriminator.save_model(sess, DIS_MODEL_FILE)
                  print ("best AUC: ", best_AUC)
                  
        buf = '\t'.join([str(AUC_actual)])
        dis_log.write(str(epoch) + '\t' + buf + '\n')
        dis_log.flush()

dis_log.close()

epoch  0 dis:  0.4983744277961383 loss:  1.539977
mejore
best AUC:  0.4983744277961383
epoch  1000 dis:  0.6146867999237415 loss:  1.6333213
mejore
best AUC:  0.6146867999237415
epoch  2000 dis:  0.6631743695496203 loss:  1.2778668
mejore
best AUC:  0.6631743695496203
epoch  3000 dis:  0.7065721448936388 loss:  2.008354
mejore
best AUC:  0.7065721448936388
epoch  4000 dis:  0.7301944191436266 loss:  1.9326353
mejore
best AUC:  0.7301944191436266
epoch  5000 dis:  0.7444025379096048 loss:  1.449018
mejore
best AUC:  0.7444025379096048
epoch  6000 dis:  0.7639117852965087 loss:  0.6821246
mejore
best AUC:  0.7639117852965087
epoch  7000 dis:  0.7801468365064264 loss:  0.49464473
mejore
best AUC:  0.7801468365064264
epoch  8000 dis:  0.7914429635391524 loss:  0.39758903
mejore
best AUC:  0.7914429635391524
epoch  9000 dis:  0.7996113301557384 loss:  0.3422217
mejore
best AUC:  0.7996113301557384
epoch  10000 dis:  0.8091242313406122 loss:  0.45051032
mejore
best AUC:  0.8091242313406122
e

1 h

In [ ]:
losses

NameError: ignored

In [ ]:
aucs

[0.49572415247587154,
 0.5662317180514732,
 0.6057415187557841,
 0.6405802112258008,
 0.6675439335645527,
 0.6913242631649663,
 0.7156699068322885,
 0.7308143598470864,
 0.7433442523143571,
 0.7519345398824647,
 0.7642779672980266,
 0.7765173203038385,
 0.7863039602763786,
 0.7969168473275811,
 0.80272771660769,
 0.8095012591546417,
 0.8147216330093032,
 0.8201110397309785,
 0.8261917472514809,
 0.8322983175197973,
 0.8342949938717108,
 0.8378111769256754,
 0.8418648976167542,
 0.8464328871552678,
 0.8501455437597926,
 0.8539392613175291,
 0.8570886662740305,
 0.8608740725484626,
 0.8607784388499193,
 0.863241103661213,
 0.8656738943065189,
 0.8692009868774703,
 0.8732967283348574,
 0.8747330631411706,
 0.8753780653590701,
 0.8767211875201576,
 0.8794115217491388,
 0.8808942039663467,
 0.8836826007118018]

1 43h

## minmax

la regularizacion es crucial

In [ ]:
BATCH_SIZE = 1000 #16
import pickle
print ("load model...")


with open( workdir+ "model_dns_IRGANvisual_boots_text.pkl", 'rb') as f: 
  param = pickle.load(f,encoding="bytes")

generator = GEN(ITEM_NUM, USER_NUM, EMB_DIM, visual_matrix, K2,reviews,lamda=0.001, param=param, initdelta=0.1,learning_rate=0.01, imageFeatureDim=1536 )

discriminator = DIS(ITEM_NUM, USER_NUM, EMB_DIM,visual_matrix, K2,reviews, lamda=0.001, param=None, initdelta=0.1,
                        learning_rate=0.01, imageFeatureDim=1536) #segundo discriminador (el que no es DNS)

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
sess.run(tf.compat.v1.global_variables_initializer())
g1= simple_test(sess, generator,user_pos_train)
d1=simple_test(sess, discriminator,user_pos_train)
print ("gen ", g1)
print ("dis ", d1)

load model...
gen  0.893905287470569
dis  0.5005801825944031


In [ ]:
simple_test(sess, generator,user_pos_train)

0.8935948956597833

In [ ]:
simple_test(sess, generator,user_pos_test)

0.8666143202968859

17 min (women)

load model...
gen  0.8837447040840262
dis  0.5011702181114173

0.8243988336222555

In [ ]:

def generate_for_d(sess, model, filename):
    data = []
    for u in user_pos_train: #para cada usario
        pos = user_pos_train[u] #sus items 
        rating = sess.run(model.all_rating, {model.u: [all_user.index(u)]}) #completar la matriz 
        rating = np.array(rating[0]) #/ 0.2  # Temperature #infla los rating #plotear que pasa con la sigmoide cuando ponemos temperatura
        exp_rating = np.exp(rating) 
        prob = exp_rating / np.sum(exp_rating)
        #print(u)
        neg = np.random.choice(list(all_items), size=len(pos), p=prob) #muestreo negativo 
        for i in range(len(pos)):
            data.append( str(all_user.index(u)) + '\t' + str(list(all_items).index(pos[i]) ) + '\t' + str(list(all_items).index(neg[i])) ) 

    with open(filename, 'w')as fout:
        fout.write('\n'.join(data))



In [ ]:
# minimax training
dis_log = open( workdir+ 'dis_log_boots_text.txt', 'w')
gen_log = open( workdir+ 'gen_log_boots_text.txt', 'w')
loss_d=[]
auc_epocs_g=[]
auc_epocs_d=[]
loss_g=[]
best_AUC=0.8505875150755186
for epoch in range(15):#15
    print("inicia minmax")
    if epoch >= 0:
        #entrena D 
         for d_epoch in range(1):##100 #para el generador fijo, entrena al discriminador
           # print("empieza a entrenarse el discriminador")
            loss_d_=0
            if d_epoch % 5 == 0:
                generate_for_d(sess, generator, DIS_TRAIN_FILE) #crea un archivo con las tripletas
                #print("pase")
                train_size = file_len(DIS_TRAIN_FILE)
            index = 1
            c=0
            while index < train_size-BATCH_SIZE: #hacer minibatch con etiquetado si es ejemplo positivo o negativo
                if index + BATCH_SIZE <= train_size + 1:
                    input_user, input_item, input_label = get_batch_data(DIS_TRAIN_FILE, index, BATCH_SIZE)
                else:
                    input_user, input_item, input_label = get_batch_data(DIS_TRAIN_FILE, index,
                                                                            train_size - index + 1)
                index += BATCH_SIZE
               # print(index)
                _, loss_dis = sess.run([discriminator.d_updates, discriminator.pre_loss],
                                 feed_dict={discriminator.u: input_user, discriminator.i: input_item, discriminator.label: input_label}) #actualizaciones de los embeddings y bias tales que clasifiquen bien
                loss_d_+=loss_dis
                c+=1
            
            loss_d.append(loss_d_/c)
            auc_actual = simple_test(sess, discriminator, user_pos_train)
            auc_epocs_d.append(auc_actual)
            print ("epoch ", epoch, "dis: ", loss_d_/c)
            if auc_actual > best_AUC:
                print ('mejore: ', auc_actual)
                best_AUC = auc_actual

         for g_epoch in range(1):  #
            #print("empieza a entrenarse el generador")
            c=0
            loss_g_=0
            for u in user_pos_train: #por cada usuario #por cada usuario
                #u= np.random.choice(list(user_pos_train.keys()),1)[0]
                sample_lambda = 0.2 #temperatura
                pos = [list(all_items).index(i) for i in user_pos_train[u]] #peliculas relevantes
                rating = sess.run(generator.all_logits, {generator.u: all_user.index(u)}) #calcula los ratings para ese usuario
                exp_rating = np.exp(rating)
                prob = exp_rating / np.sum(exp_rating)  # prob is generator distribution p_\theta
                pn = (1 - sample_lambda) * prob
                pn[pos] += sample_lambda * 1.0 / len(pos) #pedirle las prob a lo consumido + 0.2/la cantidad de items consumidos
                # Now, pn is the Pn in importance sampling, prob is generator distribution p_\theta
                sample = np.random.choice(np.arange(ITEM_NUM), 2 * len(pos), p=pn) #elige el doble de items c/r a lo consumido
                
                #               # Get reward and adapt it with importance sampling                #######
                reward = sess.run(discriminator.reward, {discriminator.u: all_user.index(u), discriminator.i: sample})
                reward = reward * prob[sample] / pn[sample]
                ######             # Update G              ####################
                _, loss_gen = sess.run([generator.gan_updates, generator.gan_loss],{generator.u: all_user.index(u), generator.i: sample, generator.reward: reward}) #actualiza pesos
                c+=1
                loss_g_+=loss_gen
                
            loss_g.append(loss_g_/c)
            auc_actual = simple_test(sess, generator, user_pos_train)
            auc_epocs_g.append(auc_actual)
            print ("epoch ", epoch, "gen: ", auc_actual)
            buf = '\t'.join([str(auc_actual) ])
            gen_log.write(str(epoch) + '\t' + buf + '\n')
            gen_log.flush()
            if auc_actual > best_AUC:
                print ('mejore: ', auc_actual)
                best_AUC = auc_actual
                generator.save_model(sess,  workdir+ "gan_generator_boots_text.pkl") #se guarda el mejor modelo, la mejor factorización matricial.

gen_log.close()
dis_log.close()



inicia minmax
epoch  0 dis:  [[7.499884  7.6103625 7.607486  ... 7.405161  7.5954103 7.3420444]]
epoch  0 gen:  0.8923883631372945
mejore:  0.8923883631372945
inicia minmax
epoch  1 dis:  [[5.580815  5.760684  5.6574054 ... 5.621743  5.5187488 5.5488544]]
epoch  1 gen:  0.890811816455097
inicia minmax
epoch  2 dis:  [[ 5.8923783 89.81277    6.9418807 ...  5.259137   6.5777464  5.1808367]]
epoch  2 gen:  0.8889771114817699
inicia minmax
epoch  3 dis:  [[819.8859  171.32182 299.53104 ... 130.37305 446.6196  143.83153]]
epoch  3 gen:  0.8868317002126466
inicia minmax
epoch  4 dis:  [[  4066.146 102955.      4611.288 ...  14886.272   5765.599  11649.335]]
epoch  4 gen:  0.884282122043487
inicia minmax
epoch  5 dis:  [[1217783.1    82652.38 2301589.8  ...  107268.73  858452.06  249192.61]]
epoch  5 gen:  0.8824415283625489
inicia minmax
epoch  6 dis:  [[53210090.   5766534.  28918004.  ...  3263820.   8563567.   2188812.8]]
epoch  6 gen:  0.8811319873955931
inicia minmax
epoch  7 dis:  [[1.

2 epoc 1 h


15 min

In [ ]:
print(simple_test(sess, discriminator,user_pos_train))
print(simple_test(sess, generator,user_pos_train))
print(simple_test(sess, discriminator,user_pos_test))
print(simple_test(sess, generator,user_pos_test))

0.5084713460441559
0.8631263960844127
0.5010876943884807
0.8708085340134709


In [ ]:
print(simple_test(sess, discriminator,user_pos_train))
print(simple_test(sess, generator,user_pos_train))
print(simple_test(sess, discriminator,user_pos_test))
print(simple_test(sess, generator,user_pos_test))


textfile = open(workdir+"IRGAN_visual_boots_text.txt", "w")
textfile.write(str(loss_d) + "\n")
textfile.write(str(auc_epocs_g) + "\n")
textfile.write(str(auc_epocs_d) + "\n")
textfile.write(str(loss_g) + "\n")
textfile.close()

0.5756609965568795
0.8806364252583635
0.5158881399868004
0.8302309605734084


0.5756609965568795
0.8806364252583635
0.5158881399868004
0.8302309605734084

In [ ]:
with open( workdir+ "gan_generator_boots_text.pkl", 'rb') as f: 
  param = pickle.load(f,encoding="bytes")

generator = GEN(ITEM_NUM, USER_NUM, EMB_DIM, visual_matrix, K2,reviews,lamda=0.001, param=param, initdelta=0.1,
                    learning_rate=0.01, imageFeatureDim=1536 )
textfile = open(workdir+"IRGAN_visual_boots.txt", "w")
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
sess.run(tf.compat.v1.global_variables_initializer())


print(simple_test(sess, generator,user_pos_train))
print(simple_test(sess, generator,user_pos_test))

0.8913130194756418
0.8681502327126349


aveces es mejor escoger el modelo final mas que el mejor en train...

In [ ]:
import torch
import torchaudio
from torchaudio import transforms

In [ ]:
print(torchaudio.info("101848-9-0-0.wav"))

AudioMetaData(sample_rate=44100, num_frames=176400, num_channels=2, bits_per_sample=16, encoding=PCM_S)


In [ ]:
sig.shape

torch.Size([2, 176400])

In [ ]:
sr

44100